In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

Error 103, reqId 64: Duplicate order id
Canceled order: Trade(contract=Stock(symbol='RHE', exchange='SMART', currency='USD'), order=StopOrder(orderId=64, clientId=13, action='SELL', totalQuantity=1, auxPrice=1.5, ocaGroup='OCA_63', parentId=63, outsideRth=True), orderStatus=OrderStatus(orderId=64, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 11, 15, 23, 46, 41, 195020, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2024, 11, 15, 23, 46, 41, 395563, tzinfo=datetime.timezone.utc), status='Cancelled', message='Error 103, reqId 64: Duplicate order id')])
Error 103, reqId 67: Duplicate order id
Canceled order: Trade(contract=Stock(symbol='RHE', exchange='SMART', currency='USD'), order=StopOrder(orderId=67, clientId=13, action='SELL', totalQuantity=1, auxPrice=1.5, ocaGroup='O

In [2]:
import trades as tr

symbol = 'RHE'
barSize = '1 min'

mkt_is_open = tr.is_open_for_trading(ib, symbol, outsideRth=True, print_status=True)
mkt_is_open 

Symbol : RHE
Exchange time now: 18:08:19
Regular market hours : 09:30:00 - 16:00:00
Extended market hours: 04:00:00 - 20:00:00


True

In [3]:
import trades as tr
import stops, entries



entrystrategy = entries.EntryStrategy(name='Hot Stuff', conditions=[ entries.PriceAbovePreviousHigh(), entries.LowerHighsPreviousBars(num_bars=2), entries.PriceNotBelowMA(ma_column='MA_cl_50', atr_column='ATR_10', atr_multiplier=0.0)])


stopStrategy = stops.StopLoss(
    init=stops.StopPrevBar(),
    # init=stops.StopGapDiff(3),
    # trail1=stops.StopPriorPiv(5),
    cond1=stops.CondDuration(5),
    trail1=stops.StopGapDiff(3),
    trail2=stops.StopPriorPiv(5),
    # trail2=stops.StopMA(21),
    cond2=stops.CondRRatio(2)
)

tx1 = tr.TradeX(ib=ib, symbol=symbol, barsize=barSize, fund_allocation=1000)

tx1.start_trade(entrystrategy, stopStrategy)
tx1.submit_bracket_order('LONG', quantity=1, stop_price=1.30,  target_price=None, outsideRth=True, limit_price=1.60)

In [13]:
from typing import Optional

def submit_bracket_order(self, direction: str, quantity: int, stop_price: float,
                        target_price: Optional[float] = None, outsideRth: bool = False,
                        limit_price: Optional[float] = None) -> None:
    """
    Submit a bracket order using ib_insync's BracketOrder tuple.
    
    Args:
        direction (str): 'LONG' or 'SHORT'
        quantity (int): Number of shares/contracts
        stop_price (float): Stop loss price
        target_price (Optional[float]): Take profit price, if None only stop loss is placed
        outsideRth (bool): Allow trading outside regular hours
        limit_price (Optional[float]): Limit price for entry, if None uses market order
    """
    from ib_insync import MarketOrder, LimitOrder, StopOrder, Order, BracketOrder
    
    if outsideRth and limit_price is None:
        raise ValueError("Limit price must be provided for orders outside regular trading hours.")
    
    def get_parent_order(action: str, quantity: int, limit_price: Optional[float] = None) -> Order:
        parent = MarketOrder(action, quantity) if limit_price is None else LimitOrder(action, quantity, limit_price)
        parent.orderId = self.ib.client.getReqId()
        parent.transmit = False  # Parent order will be transmitted last
        parent.outsideRth = outsideRth
        return parent

    def get_stop_order(action: str, quantity: int, stop_price: float, parent_id: int) -> Order:
        stop = StopOrder(action, quantity, stop_price)
        stop.orderId = self.ib.client.getReqId()
        stop.parentId = parent_id
        stop.transmit = target_price is None  # Only transmit if no target order
        stop.outsideRth = outsideRth
        return stop
    
    def get_target_order(action: str, quantity: int, target_price: float, parent_id: int) -> Order:
        take_profit = LimitOrder(action, quantity, target_price)
        take_profit.orderId = self.ib.client.getReqId()
        take_profit.parentId = parent_id
        take_profit.transmit = True  # Last order should always transmit
        take_profit.outsideRth = outsideRth
        return take_profit
    
    def place_orders(bracket: BracketOrder):
        # Place orders in sequence: parent first, then stop, then target
        trades = []
        for order in [bracket.parent, bracket.stopLoss, bracket.takeProfit]:
            if order:
                trade = self.ib.placeOrder(self.contract, order)
                trades.append(trade)
                
                if order is bracket.parent:
                    self.entry_order = trade
                    self.entry_ib_id = str(order.orderId)
                    self.entry_ib_status = trade.orderStatus.status
                elif order is bracket.stopLoss:
                    self.stop_order = trade
                    self.stop_ib_id = str(order.orderId)
                    self.stop_ib_status = trade.orderStatus.status
                elif order is bracket.takeProfit:
                    self.target_order = trade
                    self.target_ib_id = str(order.orderId)
                    self.target_ib_status = trade.orderStatus.status

        self.status = "ENTRY_SUBMITTED"

    try:
        self.entry_filled = (0, quantity)
        self.exit_filled = (0, quantity)
        
        action = 'BUY' if direction.upper() == 'LONG' else 'SELL'
        reverse_action = 'SELL' if direction.upper() == 'LONG' else 'BUY'
        
        # Create orders
        parent = get_parent_order(action, quantity, limit_price)
        stop_loss = get_stop_order(reverse_action, quantity, stop_price, parent.orderId)
        take_profit = get_target_order(reverse_action, quantity, target_price, parent.orderId) if target_price else None
        
        if take_profit:
            # Set up OCO group for stop loss and take profit
            oca_group = f"OCA_{parent.orderId}"
            stop_loss.ocaGroup = oca_group
            take_profit.ocaGroup = oca_group
            
            # Make sure transmit flags are set correctly for bracket
            parent.transmit = False
            stop_loss.transmit = False
            take_profit.transmit = True  # Last order transmits all
        else:
            # If no target order, stop loss should transmit
            parent.transmit = False
            stop_loss.transmit = True
        
        # Create bracket order tuple and place orders
        bracket = BracketOrder(parent=parent, takeProfit=take_profit, stopLoss=stop_loss)
        place_orders(bracket)

    except ValueError as ve:
        print(f"ValueError: {ve}")
    except Exception as e:
        print(f"Error submitting bracket order: {e}")

submit_bracket_order(tx1, 'LONG', 1, stop_price=1.50, target_price=1.70, outsideRth=True, limit_price=1.60)

In [14]:
from typing import List, Tuple, Optional

def submit_bracket_order(self, direction: str, quantity: int, 
                        stops: List[Tuple[float, float]], 
                        targets: Optional[List[Tuple[float, float]]] = None,
                        outsideRth: bool = False, 
                        limit_price: Optional[float] = None) -> None:
    """
    Submit a bracket order with multiple targets and stops using ib_insync.
    
    Args:
        direction (str): 'LONG' or 'SHORT'
        quantity (int): Number of shares/contracts
        stops (List[Tuple[float, float]]): List of (price, percentage) tuples for stops
                  e.g. [(1.50, 0.7), (1.45, 0.3)] means:
                  - First stop at 1.50 for 70% of position
                  - Second stop at 1.45 for 30% of position
        targets (List[Tuple[float, float]], optional): List of (price, percentage) tuples for targets
                  e.g. [(1.70, 0.5), (1.80, 0.5)] means:
                  - First target at 1.70 for 50% of position
                  - Second target at 1.80 for 50% of position
        outsideRth (bool): Allow trading outside regular hours
        limit_price (Optional[float]): Limit price for entry, if None uses market order
    
    Example:
        # Two targets and two stops
        broker.submit_bracket_order(
            direction='LONG',
            quantity=100,
            stops=[(1.50, 0.7), (1.45, 0.3)],
            targets=[(1.70, 0.6), (1.80, 0.4)],
            limit_price=1.60
        )
    """
    from ib_insync import MarketOrder, LimitOrder, StopOrder, Order
    from decimal import Decimal, ROUND_DOWN
    
    if outsideRth and limit_price is None:
        raise ValueError("Limit price must be provided for orders outside regular trading hours.")
    
    def validate_percentages(orders: List[Tuple[float, float]], name: str) -> None:
        """Validate that percentages sum to 1.0"""
        total = sum(pct for _, pct in orders)
        if not 0.99 <= total <= 1.01:
            raise ValueError(f"{name} percentages must sum to 1.0 (got {total})")
    
    # Validate both stops and targets
    validate_percentages(stops, "Stop")
    if targets:
        validate_percentages(targets, "Target")
    
    def get_parent_order(action: str, quantity: int, limit_price: Optional[float] = None) -> Order:
        parent = MarketOrder(action, quantity) if limit_price is None else LimitOrder(action, quantity, limit_price)
        parent.orderId = self.ib.client.getReqId()
        parent.transmit = False
        parent.outsideRth = outsideRth
        return parent

    def get_stop_order(action: str, quantity: int, stop_price: float, parent_id: int,
                      is_last_order: bool) -> Order:
        stop = StopOrder(action, quantity, stop_price)
        stop.orderId = self.ib.client.getReqId()
        stop.parentId = parent_id
        stop.transmit = is_last_order and not targets  # Only transmit if last order and no targets
        stop.outsideRth = outsideRth
        return stop
    
    def get_target_order(action: str, quantity: int, target_price: float, parent_id: int,
                        is_last_order: bool) -> Order:
        take_profit = LimitOrder(action, quantity, target_price)
        take_profit.orderId = self.ib.client.getReqId()
        take_profit.parentId = parent_id
        take_profit.transmit = is_last_order  # Only last order transmits
        take_profit.outsideRth = outsideRth
        return take_profit
    
    def calculate_quantities(total_quantity: int, percentages: List[float]) -> List[int]:
        """Calculate integer quantities for each order while handling rounding."""
        quantities = []
        remaining = total_quantity
        
        for i, pct in enumerate(percentages[:-1]):
            qty = int(Decimal(str(total_quantity * pct)).quantize(Decimal('1'), rounding=ROUND_DOWN))
            quantities.append(qty)
            remaining -= qty
            
        quantities.append(remaining)  # Last quantity gets remainder
        return quantities

    try:
        self.entry_filled = (0, quantity)
        self.exit_filled = (0, quantity)
        
        action = 'BUY' if direction.upper() == 'LONG' else 'SELL'
        reverse_action = 'SELL' if direction.upper() == 'LONG' else 'BUY'
        
        # Create parent order
        parent = get_parent_order(action, quantity, limit_price)
        
        # Initialize lists to store orders and trades
        all_orders = [parent]
        self.stop_orders = []
        self.stop_ib_ids = []
        self.stop_ib_statuses = []
        self.target_orders = []
        self.target_ib_ids = []
        self.target_ib_statuses = []
        
        # Create OCO group for all exit orders
        oca_group = f"OCA_{parent.orderId}"
        
        # Process stop orders
        stop_prices, stop_percentages = zip(*stops)
        stop_quantities = calculate_quantities(quantity, stop_percentages)
        
        for i, (stop_price, stop_qty) in enumerate(zip(stop_prices, stop_quantities)):
            is_last_stop = (i == len(stops) - 1)
            stop_loss = get_stop_order(
                reverse_action, stop_qty, stop_price,
                parent.orderId, is_last_stop
            )
            stop_loss.ocaGroup = oca_group
            all_orders.append(stop_loss)
            self.stop_orders.append(stop_loss)
        
        # Process target orders if specified
        if targets:
            target_prices, target_percentages = zip(*targets)
            target_quantities = calculate_quantities(quantity, target_percentages)
            
            for i, (target_price, target_qty) in enumerate(zip(target_prices, target_quantities)):
                is_last_order = (i == len(targets) - 1)
                take_profit = get_target_order(
                    reverse_action, target_qty, target_price,
                    parent.orderId, is_last_order
                )
                take_profit.ocaGroup = oca_group
                all_orders.append(take_profit)
                self.target_orders.append(take_profit)
        
        # Place all orders
        for i, order in enumerate(all_orders):
            trade = self.ib.placeOrder(self.contract, order)
            
            if order is parent:
                self.entry_order = trade
                self.entry_ib_id = str(order.orderId)
                self.entry_ib_status = trade.orderStatus.status
            elif order in self.stop_orders:
                self.stop_ib_ids.append(str(order.orderId))
                self.stop_ib_statuses.append(trade.orderStatus.status)
            else:
                self.target_ib_ids.append(str(order.orderId))
                self.target_ib_statuses.append(trade.orderStatus.status)
        
        self.status = "ENTRY_SUBMITTED"

    except ValueError as ve:
        print(f"ValueError: {ve}")
    except Exception as e:
        print(f"Error submitting bracket order: {e}")

In [16]:
# Tight stop for most of position, wider stop for remainder
submit_bracket_order(
    tx1,
    direction='LONG',
    quantity=1000,
    stops=[(1.50, 0.8), (1.40, 0.2)],  # 80% tight stop, 20% wide stop
    targets=[(1.70, 1.0)],             # Single target for full position
    limit_price=1.60
)

# # Multiple stops and targets for scaling
# submit_bracket_order(
#     direction='LONG',
#     quantity=1000,
#     stops=[(1.50, 0.5), (1.45, 0.5)],    # Split stop levels
#     targets=[(1.70, 0.3), (1.80, 0.4), (1.90, 0.3)],  # Three targets
#     limit_price=1.60
# )

# # Just multiple stops
# submit_bracket_order(
#     direction='LONG',
#     quantity=1000,
#     stops=[(1.50, 0.4), (1.45, 0.4), (1.40, 0.2)],  # Three stop levels
#     limit_price=1.60
# )